In [103]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math

In [104]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [105]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')

        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.86,1.34,38.25,0,Consumer Cyclical,Specialty Retail,Denmark,20.63,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.76,1.30,43.28,0,Technology,Information Technology Services,Switzerland,12.48,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.13,1.12,-0.94,0,Industrials,Aerospace & Defense,United Kingdom,21.62,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.06,0.06,0.00,0,Industrials,Aerospace & Defense,Germany,100.20,5.97
SAABF,SAABF,Saab AB,Yes,0.38,0.38,0.00,0,Industrials,Aerospace & Defense,Sweden,47.84,3.70
BCKIY,BCKIY,Babcock International Group Plc,No,1.02,1.02,0.00,0,Industrials,Engineering & Construction,United Kingdom,19.15,5.08
BAESY,BAESY,BAE Systems PLC,No,0.83,0.77,-5.18,0,Industrials,Aerospace & Defense,United Kingdom,25.00,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.25,1.06,-15.38,0,Technology,Communication Equipment,Sweden,63.34,20.30
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.56,0.56,0.00,0,Industrials,Aerospace & Defense,Norway,55.87,6.66
GMAB,GMAB,Genmab A/S,Yes,1.64,2.22,46.19,0,Healthcare,Biotechnology,Denmark,12.49,10.91


Got data for 32 stocks


In [106]:
print(yf.Ticker('RNMBY').info.keys())

dict_keys(['address1', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'tradeable', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'heldPercentInsiders', 'heldPer

In [107]:
yf.Ticker('MATAS.CO').history(period='10y')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-03-11 00:00:00+01:00,103.466243,104.477314,101.781125,102.118149,104504,0.0,0.0
2015-03-12 00:00:00+01:00,102.118159,102.792206,101.781135,102.792206,223198,0.0,0.0
2015-03-13 00:00:00+01:00,102.455177,103.466248,101.781130,102.455177,100930,0.0,0.0
2015-03-16 00:00:00+01:00,102.455167,102.455167,101.107073,101.781120,380850,0.0,0.0
2015-03-17 00:00:00+01:00,101.444106,102.792201,101.444106,102.455177,66273,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-05 00:00:00+01:00,137.600006,139.399994,136.399994,136.600006,70726,0.0,0.0
2025-03-06 00:00:00+01:00,136.600006,137.000000,132.800003,133.600006,59365,0.0,0.0
2025-03-07 00:00:00+01:00,133.600006,134.199997,131.399994,133.199997,62538,0.0,0.0


In [108]:
cashflow = yf.Ticker('MATAS.CO').cashflow
earning_dates = cashflow.columns.tolist()
print(earning_dates)

yf.Ticker('MATAS.CO').history(period='10y')

[Timestamp('2024-03-31 00:00:00'), Timestamp('2023-03-31 00:00:00'), Timestamp('2022-03-31 00:00:00'), Timestamp('2021-03-31 00:00:00'), Timestamp('2020-03-31 00:00:00')]


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-03-11 00:00:00+01:00,103.466243,104.477314,101.781125,102.118149,104504,0.0,0.0
2015-03-12 00:00:00+01:00,102.118159,102.792206,101.781135,102.792206,223198,0.0,0.0
2015-03-13 00:00:00+01:00,102.455177,103.466248,101.781130,102.455177,100930,0.0,0.0
2015-03-16 00:00:00+01:00,102.455167,102.455167,101.107073,101.781120,380850,0.0,0.0
2015-03-17 00:00:00+01:00,101.444106,102.792201,101.444106,102.455177,66273,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-05 00:00:00+01:00,137.600006,139.399994,136.399994,136.600006,70726,0.0,0.0
2025-03-06 00:00:00+01:00,136.600006,137.000000,132.800003,133.600006,59365,0.0,0.0
2025-03-07 00:00:00+01:00,133.600006,134.199997,131.399994,133.199997,62538,0.0,0.0


In [109]:
yf.Ticker(symbols[0]).cashflow

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Free Cash Flow,239000000.0,424000000.0,327100000.0,803600000.0,NaN
Repurchase Of Capital Stock,-21000000.0,0.0,-75100000.0,0.0,NaN
Repayment Of Debt,-189000000.0,-126000000.0,-708400000.0,-725000000.0,NaN
Issuance Of Debt,1121000000.0,0.0,763500000.0,0.0,NaN
Capital Expenditure,-406000000.0,-254000000.0,-183400000.0,-148400000.0,NaN
End Cash Position,131000000.0,37000000.0,28200000.0,40700000.0,NaN
Beginning Cash Position,37000000.0,28000000.0,40700000.0,106600000.0,NaN
Effect Of Exchange Rate Changes,4000000.0,0.0,NaN,NaN,NaN
Changes In Cash,90000000.0,9000000.0,-12500000.0,-65900000.0,NaN
Financing Cash Flow,466000000.0,-413000000.0,-291400000.0,-840000000.0,NaN


In [110]:
yf.Ticker(symbols[0]).balancesheet

,2024-03-31,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Treasury Shares Number,382981.0,358423.0,626585.0,22943.0,NaN
Ordinary Shares Number,37908511.0,37933069.0,37664907.0,38268549.0,NaN
Share Issued,38291492.0,38291492.0,38291492.0,38291492.0,NaN
Net Debt,1931000000.0,991000000.0,1125800000.0,1057600000.0,NaN
Total Debt,3272000000.0,1679000000.0,1677000000.0,1767900000.0,NaN
...,...,...,...,...,...
Accounts Receivable,76000000.0,44000000.0,26600000.0,15200000.0,NaN
Allowance For Doubtful Accounts Receivable,0.0,-1000000.0,NaN,NaN,NaN
Gross Accounts Receivable,76000000.0,45000000.0,NaN,NaN,NaN
Cash Cash Equivalents And Short Term Investments,131000000.0,37000000.0,28200000.0,40700000.0,NaN


In [117]:
symbols = ['AAPL']

In [118]:
df = pd.DataFrame()
for symbol in symbols:
    print(f'{symbol}...')

    ticker = yf.Ticker(symbol)

    earning_dates = ticker.cashflow.columns.tolist()

    quant_data = ticker.history(period='10y')

    cashflow = ticker.cashflow
    cashflow_columns = cashflow[earning_dates[0]].keys().tolist()

    balancesheet = ticker.balancesheet
    balancesheet_columns = balancesheet[earning_dates[0]].keys().tolist()

    for earning_date in earning_dates:
        row_data = {'Ticker': symbol, 'Date': earning_date}
        


        for column in cashflow_columns:
            row_data[column] = cashflow[earning_date][column]

        for column in balancesheet_columns:
            row_data[column] = balancesheet[earning_date][column]
        
        df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)
        
df.to_csv(result_folder / 'earnings_data.csv', index=False)

AAPL...
